In [ ]:
!pip install tabulate

In [ ]:
!pip install openpyxl

In [ ]:
!pip install sqlalchemy pymysql

In [ ]:
!pip install watchdog

In [1]:
import os
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Path folder yang dipantau
folder_path = 'C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel'

# Daftar sheet yang ingin diekstrak
sheets_to_extract = ['1000000', '1210000', '1311000', '1510000']

# Fungsi untuk memproses dan mengekspor data ke MySQL
def process_and_export(file_path):
    # DataFrame gabungan
    combined_data = pd.DataFrame(columns=['Keterangan', 'Value', 'Emiten'])

    # Extract Kode Entitas dari nama file
    kode_entitas = os.path.splitext(os.path.basename(file_path))[0].split('-')[-1]

    try:
        # Baca file Excel
        excel_file = pd.ExcelFile(file_path)

        # Loop untuk membaca hanya sheet yang dipilih
        for sheet in sheets_to_extract:
            if sheet in excel_file.sheet_names:  # Mengecek keberadaan sheet pada file
                print(f"Sheet '{sheet}' ditemukan dalam file {file_path}, memproses data...")
                data = excel_file.parse(sheet)

                # Drop 3 baris pertama
                data = data.iloc[3:]

                # Hanya mengambil 2 kolom pertama, jika ada
                data_selected = data.iloc[:, :2]

                # Mengganti nama kolom menjadi 'Keterangan' dan 'Value'
                data_selected.columns = ['Keterangan', 'Value']

                # Menambahkan kolom 'Emiten' dengan nilai kode entitas
                data_selected['Emiten'] = kode_entitas

                # Menggabungkan ke DataFrame utama
                combined_data = pd.concat([combined_data, data_selected], ignore_index=True)

        # Pindahkan kolom 'Emiten' ke posisi pertama
        columns_order = ['Emiten', 'Keterangan', 'Value']
        combined_data = combined_data[columns_order]

        # Mengganti nama kolom menjadi sesuai dengan nama kolom di MySQL
        combined_data.columns = ['emiten_excel', 'keterangan_excel', 'value_excel']

        # Pengaturan koneksi MySQL
        mysql_user = 'root'  # Ganti dengan username MySQL Anda
        mysql_password = 'Dika007!#'  # Ganti dengan password MySQL Anda
        mysql_host = 'localhost'  # Host MySQL, misalnya localhost
        mysql_port = '3306'  # Port MySQL, default 3306
        db_name = 'db_calk'  # Nama database MySQL yang ingin digunakan

        # Membuat koneksi ke MySQL menggunakan SQLAlchemy
        engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{db_name}')

        # Mengekspor DataFrame ke MySQL
        combined_data.to_sql('tb_excel', con=engine, if_exists='append', index=False)

        print(f"\nData dari {file_path} telah berhasil disisipkan ke tabel 'tb_excel' dalam database '{db_name}'.")

        # Menandai file sudah diproses dengan membuat file .processed
        processed_file_path = file_path + '.processed'
        with open(processed_file_path, 'w') as f:
            f.write('Processed')  # Anda dapat menambahkan informasi lain jika diperlukan

    except Exception as e:
        print(f"Error saat memproses file {file_path}: {e}")

# Kelas untuk menangani event file baru
class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return  # Skip folder
        if event.src_path.endswith(('.xlsx', '.xls')):  # Pastikan hanya file Excel yang diproses
            processed_file = event.src_path + '.processed'
            if not os.path.exists(processed_file):  # Hanya memproses jika file belum diproses
                print(f"\nFile baru ditemukan: {event.src_path}")
                process_and_export(event.src_path)
            else:
                print(f"File {event.src_path} sudah diproses sebelumnya, melewatkan...")

# Fungsi untuk memproses semua file yang sudah ada
def process_existing_files():
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith(('.xlsx', '.xls')):  # Pastikan hanya file Excel yang diproses
            processed_file = file_path + '.processed'
            if not os.path.exists(processed_file):  # Hanya memproses jika file belum diproses
                print(f"\nMemproses file yang sudah ada: {file_path}")
                process_and_export(file_path)
            else:
                print(f"File {file_path} sudah diproses sebelumnya, melewatkan...")

# Membuat observer untuk memantau folder
observer = Observer()
event_handler = MyHandler()
observer.schedule(event_handler, folder_path, recursive=False)

# Proses file yang sudah ada di folder
process_existing_files()

# Mulai memantau folder
observer.start()
print(f"Memantau folder: {folder_path} untuk file baru...")

try:
    while True:
        pass  # Program akan terus berjalan dan memantau folder
except KeyboardInterrupt:
    observer.stop()
    print("\nPemantauan dihentikan.")
observer.join()


Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx
Sheet '1000000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx telah berhasil disisipkan ke tabel 'tb_excel' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_22608\4222663142.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Emiten'] = kode_entitas
C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_22608\4222663142.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Emiten'] = kode_enti

Sheet '1000000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx, memproses data...
Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx telah berhasil disisipkan ke tabel 'tb_excel' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_22608\4222663142.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Emiten'] = kode_entitas
C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_22608\4222663142.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Emiten'] = kode_enti

Sheet '1000000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...
Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx telah berhasil disisipkan ke tabel 'tb_excel' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_22608\4222663142.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Emiten'] = kode_entitas
C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_22608\4222663142.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Emiten'] = kode_enti

Sheet '1000000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx telah berhasil disisipkan ke tabel 'tb_excel' dalam database 'db_calk'.
Memantau folder: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel untuk file baru...


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)
C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_22608\4222663142.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Emiten'] = kode_entitas
C:\Users\MSI KATANA\AppData\Local\Temp\ipykernel_22608\4222663142.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_selected['Emiten'] = kode_enti


Pemantauan dihentikan.


In [ ]:
import os
import pandas as pd
from sqlalchemy import create_engine
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Path folder yang dipantau
folder_path = 'C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel'

# Daftar sheet yang ingin diekstrak
sheets_to_extract = ['1210000', '1311000', '1510000']

# Fungsi untuk memproses dan mengekspor data ke MySQL
def process_and_export(file_path):
    # DataFrame gabungan
    combined_data = pd.DataFrame(columns=['keterangan', 'value', 'emiten', 'nama_perusahaan', 'kuartal'])

    try:
        # Membaca file Excel
        excel_file = pd.ExcelFile(file_path)

        # Ekstrak 'nama_perusahaan' dan 'kuartal' dari sheet '1000000'
        nama_perusahaan = None
        kuartal = None
        if '1000000' in excel_file.sheet_names:
            sheet_1000000 = excel_file.parse('1000000')
            
            # Memastikan file memiliki setidaknya 2 baris pertama untuk ekstraksi
            if len(sheet_1000000) >= 2:
                nama_perusahaan = sheet_1000000.iloc[4, 1]
                kuartal = sheet_1000000.iloc[19, 1]
                emiten = sheet_1000000.iloc[6, 1]       

        # Loop untuk membaca sheet yang dipilih
        for sheet in sheets_to_extract:
            if sheet in excel_file.sheet_names:  # Mengecek keberadaan sheet pada file
                print(f"Sheet '{sheet}' ditemukan dalam file {file_path}, memproses data...")
                data = excel_file.parse(sheet)

                # Drop 3 baris pertama
                data = data.iloc[2:]

                # Hanya mengambil 2 kolom pertama, jika ada
                data_selected = data.iloc[:, :2]

                # Mengganti nama kolom menjadi 'keterangan' dan 'value'
                data_selected.columns = ['keterangan', 'value']

                # Menambahkan kolom 'Emiten' dengan nilai 'emiten'
                data_selected['emiten'] = emiten

                # Tambahkan kolom 'nama_perusahaan' dan 'kuartal'
                data_selected['nama_perusahaan'] = nama_perusahaan
                data_selected['kuartal'] = kuartal

                # Menggabungkan ke DataFrame utama
                combined_data = pd.concat([combined_data, data_selected], ignore_index=True)


        # Pindahkan kolom ke posisi sesuai kebutuhan
        columns_order = ['emiten', 'nama_perusahaan', 'kuartal', 'keterangan', 'value']
        combined_data = combined_data[columns_order]

        # Mengganti nama kolom menjadi sesuai dengan nama kolom di MySQL
        combined_data.columns = ['emiten', 'nama_perusahaan', 'kuartal', 'keterangan', 'value', ]

        # Pengaturan koneksi MySQL
        mysql_user = 'root'  # Ganti dengan username MySQL Anda
        mysql_password = 'Dika007!#'  # Ganti dengan password MySQL Anda
        mysql_host = 'localhost'  # Host MySQL, misalnya localhost
        mysql_port = '3306'  # Port MySQL, default 3306
        db_name = 'db_calk'  # Nama database MySQL yang ingin digunakan

        # Membuat koneksi ke MySQL menggunakan SQLAlchemy
        engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{db_name}')

        # Mengekspor DataFrame ke MySQL
        combined_data.to_sql('tb_calk', con=engine, if_exists='append', index=False)

        print(f"\nData dari {file_path} telah berhasil disisipkan ke tabel 'tb_excel' dalam database '{db_name}'.")

        # Menandai file sudah diproses dengan membuat file .processed
        processed_file_path = file_path + '.processed'
        with open(processed_file_path, 'w') as f:
            f.write('Processed')  # Anda dapat menambahkan informasi lain jika diperlukan

    except Exception as e:
        print(f"Error saat memproses file {file_path}: {e}")

# Kelas untuk menangani event file baru
class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return  # Skip folder
        if event.src_path.endswith(('.xlsx', '.xls')):  # Pastikan hanya file Excel yang diproses
            processed_file = event.src_path + '.processed'
            if not os.path.exists(processed_file):  # Hanya memproses jika file belum diproses
                print(f"\nFile baru ditemukan: {event.src_path}")
                process_and_export(event.src_path)
            else:
                print(f"File {event.src_path} sudah diproses sebelumnya, melewatkan...")

# Fungsi untuk memproses semua file yang sudah ada
def process_existing_files():
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith(('.xlsx', '.xls')):  # Pastikan hanya file Excel yang diproses
            processed_file = file_path + '.processed'
            if not os.path.exists(processed_file):  # Hanya memproses jika file belum diproses
                print(f"\nMemproses file yang sudah ada: {file_path}")
                process_and_export(file_path)
            else:
                print(f"File {file_path} sudah diproses sebelumnya, melewatkan...")

# Membuat observer untuk memantau folder
observer = Observer()
event_handler = MyHandler()
observer.schedule(event_handler, folder_path, recursive=False)

# Proses file yang sudah ada di folder
process_existing_files()

# Mulai memantau folder
observer.start()
print(f"Memantau folder: {folder_path} untuk file baru...")

try:
    while True:
        pass  # Program akan terus berjalan dan memantau folder
except KeyboardInterrupt:
    observer.stop()
    print("\nPemantauan dihentikan.")
observer.join()

In [3]:
import os
import pandas as pd
from sqlalchemy import create_engine
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler

# Path folder yang dipantau
folder_path = 'C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel'

# Daftar sheet yang ingin diekstrak
sheets_to_extract = ['1210000', '1311000', '1510000']

# Fungsi untuk memproses dan mengekspor data ke MySQL
def process_and_export(file_path):
    # DataFrame gabungan
    combined_data = pd.DataFrame(columns=['keterangan', 'value', 'emiten', 'nama_perusahaan', 'kuartal', 'item'])

    try:
        # Membaca file Excel
        excel_file = pd.ExcelFile(file_path)

        # Ekstrak 'nama_perusahaan' dan 'kuartal' dari sheet '1000000'
        nama_perusahaan = None
        kuartal = None
        if '1000000' in excel_file.sheet_names:
            sheet_1000000 = excel_file.parse('1000000')
            
            # Memastikan file memiliki setidaknya 2 baris pertama untuk ekstraksi
            if len(sheet_1000000) >= 2:
                nama_perusahaan = sheet_1000000.iloc[4, 1]
                kuartal = sheet_1000000.iloc[19, 1]
                emiten = sheet_1000000.iloc[6, 1]       

        # Loop untuk membaca sheet yang dipilih
        for sheet in sheets_to_extract:
            if sheet in excel_file.sheet_names:  # Mengecek keberadaan sheet pada file
                print(f"Sheet '{sheet}' ditemukan dalam file {file_path}, memproses data...")
                data = excel_file.parse(sheet)

                # Drop 3 baris pertama
                data = data.iloc[3:]

                # Hanya mengambil 2 kolom pertama, jika ada
                data_selected = data.iloc[1:, :2]

                # Mengganti nama kolom menjadi 'keterangan' dan 'value'
                data_selected.columns = ['keterangan', 'value']

                # Menambahkan kolom 'Emiten' dengan nilai 'emiten'
                data_selected['emiten'] = emiten

                # Menambahkan kolom 'nama_perusahaan' dan 'kuartal'
                data_selected['nama_perusahaan'] = nama_perusahaan
                data_selected['kuartal'] = kuartal

                # Menambahkan kolom 'item' dengan nilai dari baris ke-4 (indeks 3)
                data_selected['item'] = data.iloc[0, 0]  # Mengambil nilai pada baris ke-4, kolom pertama

                # Menggabungkan ke DataFrame utama
                combined_data = pd.concat([combined_data, data_selected], ignore_index=True)

        # Pindahkan kolom ke posisi sesuai kebutuhan
        columns_order = ['emiten', 'nama_perusahaan', 'kuartal', 'item', 'keterangan', 'value']
        combined_data = combined_data[columns_order]

        # Mengganti nama kolom menjadi sesuai dengan nama kolom di MySQL
        combined_data.columns = ['emiten', 'nama_perusahaan', 'kuartal', 'item', 'keterangan', 'value']

        # Pengaturan koneksi MySQL
        mysql_user = 'root'  # Ganti dengan username MySQL Anda
        mysql_password = 'Dika007!#'  # Ganti dengan password MySQL Anda
        mysql_host = 'localhost'  # Host MySQL, misalnya localhost
        mysql_port = '3306'  # Port MySQL, default 3306
        db_name = 'db_calk'  # Nama database MySQL yang ingin digunakan

        # Membuat koneksi ke MySQL menggunakan SQLAlchemy
        engine = create_engine(f'mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/{db_name}')

        # Mengekspor DataFrame ke MySQL
        combined_data.to_sql('tb_calk', con=engine, if_exists='append', index=False)

        print(f"\nData dari {file_path} telah berhasil disisipkan ke tabel 'tb_excel' dalam database '{db_name}'.")

        # Menandai file sudah diproses dengan membuat file .processed
        processed_file_path = file_path + '.processed'
        with open(processed_file_path, 'w') as f:
            f.write('Processed')  # Anda dapat menambahkan informasi lain jika diperlukan

    except Exception as e:
        print(f"Error saat memproses file {file_path}: {e}")

# Kelas untuk menangani event file baru
class MyHandler(FileSystemEventHandler):
    def on_created(self, event):
        if event.is_directory:
            return  # Skip folder
        if event.src_path.endswith(('.xlsx', '.xls')):  # Pastikan hanya file Excel yang diproses
            processed_file = event.src_path + '.processed'
            if not os.path.exists(processed_file):  # Hanya memproses jika file belum diproses
                print(f"\nFile baru ditemukan: {event.src_path}")
                process_and_export(event.src_path)
            else:
                print(f"File {event.src_path} sudah diproses sebelumnya, melewatkan...")

# Fungsi untuk memproses semua file yang sudah ada
def process_existing_files():
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        if file_path.endswith(('.xlsx', '.xls')):  # Pastikan hanya file Excel yang diproses
            processed_file = file_path + '.processed'
            if not os.path.exists(processed_file):  # Hanya memproses jika file belum diproses
                print(f"\nMemproses file yang sudah ada: {file_path}")
                process_and_export(file_path)
            else:
                print(f"File {file_path} sudah diproses sebelumnya, melewatkan...")

# Membuat observer untuk memantau folder
observer = Observer()
event_handler = MyHandler()
observer.schedule(event_handler, folder_path, recursive=False)

# Proses file yang sudah ada di folder
process_existing_files()

# Mulai memantau folder
observer.start()
print(f"Memantau folder: {folder_path} untuk file baru...")

try:
    while True:
        pass  # Program akan terus berjalan dan memantau folder
except KeyboardInterrupt:
    observer.stop()
    print("\nPemantauan dihentikan.")
observer.join()



Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ABBA.xlsx telah berhasil disisipkan ke tabel 'tb_excel' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACES.xlsx telah berhasil disisipkan ke tabel 'tb_excel' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-ACRO.xlsx telah berhasil disisipkan ke tabel 'tb_excel' dalam database 'db_calk'.

Memproses file yang sudah ada: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx


c:\Users\MSI KATANA\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


Sheet '1210000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1311000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...
Sheet '1510000' ditemukan dalam file C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx, memproses data...

Data dari C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel\FinancialStatement-2024-I-AISA.xlsx telah berhasil disisipkan ke tabel 'tb_excel' dalam database 'db_calk'.
Memantau folder: C:/TI-Dwika/Semester 7/Pangkalan Data/file_excel untuk file baru...

Pemantauan dihentikan.
